# Song-based Lyric Generator

In [1]:
# char and word works ok-

# models need to be saved

# some code sections, such as generator can be generalized

# NB: Several sections of code are copied, modify

In [13]:
!pip install pycontractions

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\bleli\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\bleli\\AppData\\Local\\Temp\\pip-install-2v0rkxww\\pyemd_e336b2f918ab4f70973773506f1071cb\\setup.py'"'"'; __file__='"'"'C:\\Users\\bleli\\AppData\\Local\\Temp\\pip-install-2v0rkxww\\pyemd_e336b2f918ab4f70973773506f1071cb\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\bleli\AppData\Local\Temp\pip-wheel-s2_5ov7e'
       cwd: C:\Users\bleli\AppData\Local\Temp\pip-install-2v0rkxww\pyemd_e336b2f918ab4f70973773506f1071cb\
  Complete output (11 lines):
  running bdist_wheel
  running buildCollecting language-check>=1.0
  Created wheel for language-check: filename=language_check-1.1-py3-none-any.whl s

In [1]:
#@title Libraries
#!pip install tensorflow
#!pip install nltk
#!pip install contractions
#nltk.download('punkt')
import re
import os
import io
import sys
import time
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
from pycontractions import contractions

print('Tensorflow Version: ', tf.__version__)

ModuleNotFoundError: No module named 'pycontractions'

In [20]:
import pycontractions

ModuleNotFoundError: No module named 'pycontractions'

In [6]:
df = pd.read_csv('cleaned_dataset.csv')
df.head(5)

,Unnamed: 0,Unnamed: 0.1,title,date,album,lyrics,intro,verse,chorus,bridge,outro,intro_cleaned,verse_cleaned,bridge_cleaned,chorus_cleaned,outro_cleaned
0,0,0,Let It Be,"May 8, 1970",Let It Be,\r\n\r\n[Verse 1]\r\nWhen I find myself in tim...,[],['[Verse 1]\nWhen I find myself in times of tr...,"['[Chorus]\nLet it be, let it be, let it be, l...",[],[],NaN,"when i find myself in times of trouble, mothe...",NaN,"let it be, let it be, let it be, let it be wh...",NaN
1,1,1,Come Together,"September 26, 1969",Abbey Road,\r\n\r\n[Intro]\r\nShoot me\r\nShoot me\r\nSho...,['[Intro]\nShoot me\nShoot me\nShoot me\nShoot...,"[""[Verse 1]\nHere come old flat-top, he come g...","['[Chorus]\nCome together, right now\nOver me'...",[],"['[Outro]\nCome together, yeah\nCome together,...",shoot me shoot me shoot me shoot me',"here come old flat-top, he come groovin' up s...",NaN,"come together, right now over me', come toget...","come together, yeah come together, yeah come ..."
2,2,2,Yesterday,"September 13, 1965",Help! (UK),\r\n\r\n[Verse 1]\r\nYesterday\r\nAll my troub...,[],"[""[Verse 1]\nYesterday\nAll my troubles seemed...","[""[Chorus]\nWhy she had to go\nI don't know, s...",[],"['[Outro]\nMmm-mmm-mmm-mmm-mmm, hmm-hmm']",NaN,yesterday all my troubles seemed so far away ...,NaN,"why she had to go i don't know, she wouldn't ...","mmm-mmm-mmm-mmm-mmm, hmm-hmm'"
3,3,3,Something,"September 26, 1969",Abbey Road,\r\n\r\n[Verse 1]\r\nSomething in the way she ...,[],['[Verse 1]\nSomething in the way she moves\nA...,"[""[Chorus]\nI don't want to leave her now\nYou...","[""[Bridge]\nYou're asking me will my love grow...","[""[Outro]\nI don't want to leave her now\nYou ...",NaN,something in the way she moves attracts me li...,you're asking me will my love grow i don't kn...,i don't want to leave her now you know i beli...,i don't want to leave her now you know i beli...
4,4,4,Here Comes the Sun,"September 26, 1969",Abbey Road,"\r\n\r\n[Chorus]\r\nHere comes the sun, doo da...",[],"[""[Verse 1]\nLittle darling, it's been a long ...","[""[Chorus]\nHere comes the sun, doo da doo doo...","['[Bridge]\nSun, sun, sun, here it comes\nSun,...","[""[Outro]\nHere comes the sun, doo da doo doo\...",NaN,"little darling, it's been a long cold lonely ...","sun, sun, sun, here it comes sun, sun, sun, h...","here comes the sun, doo da doo doo here comes...","here comes the sun, doo da doo doo here comes..."


**Clean Lyrics**

In [16]:
#@title Clean Lyrics
def clean(s):
    
    # Expand contractions
    s = s.split()
    #new_s = []
    #for word in s:
        #new_s.append(contractions.fix(word))
    #s = " ".join(new_s)

    # Format words and remove unwanted characters
    s = re.sub(r'[\(\[].*?[\)\]] +', '', s)
    s = re.sub(r' [\(\[].*?[\)\]]', '', s)
    s = re.sub(r"[^’'a-zA-Z \ ]+", ' ', s)
    s = re.sub(' +', ' ', s)
    s = s.lower()
    return s

In [15]:
# new colunn with clean lyrics
df['lyrics_cleaned'] = list(map(clean, df.lyrics))
df['test'] = df['lyrics_cleaned'][310:311]
df['lyrics_cleaned'] = df['lyrics_cleaned'][0:310]
lyrics = df['lyrics_cleaned'].dropna()
test = df['test'].dropna()
test.reset_index(drop = True, inplace=True)

TypeError: expected string or bytes-like object

In [183]:
#@title Extract Text
# extract all from lyrics cleaned col to string
# todo: regex remove all stopchars, puncation etc, except: [Intro], [Verse], [Chorus], hence remove verse number ([Verse 1])
def extract_text(lyrics):
    global corpus
    lst = []
    for i in range(len(lyrics)):
        lst.append(str(lyrics[i]))
    data = ''.join(lst)
    
    corpus = []
    corpus += [w for w in data.split(' ') if w.strip() != '' or w == '\n']
    lyrics = ' '.join(corpus)
    
    return lyrics

In [232]:
from collections import Counter
import random

# Expand contractions
def expand(s): 
    s = s.lower()
    s = s.split()
    new_s = []
    for word in s:
        new_s.append(contractions.fix(word))
    s = " ".join(new_s)
    return s

lyrics = extract_text(lyrics)
#lyrics = expand(lyrics)
intro = extract_text(df['intro_cleaned'])
#intro = expand(intro)
verse = extract_text(df['verse_cleaned'])
#verse = expand(verse)
chorus = extract_text(df['chorus_cleaned'])
#chorus = expand(chorus)
bridge = extract_text(df['bridge_cleaned'])
#bridge = expand(bridge)
outro = extract_text(df['outro_cleaned'])
#outro = expand(outro)
test = extract_text(test)
#test = expand(test)

# most common words from string
Counter_intro = Counter(intro.split())
most_occur_intro = Counter_intro.most_common(10)

Counter_verse = Counter(verse.split())
most_occur_verse = Counter_verse.most_common(20)

Counter_chorus = Counter(chorus.split())
most_occur_chorus = Counter_chorus.most_common(20)

Counter_bridge = Counter(bridge.split())
most_occur_bridge = Counter_bridge.most_common(20)

Counter_outro = Counter(outro.split())
most_occur_outro = Counter_outro.most_common(10)

#random.choice(most_occur_verse)[0]

**Get Vocabulary**

In [133]:
# Choose characted based or word based text generation
char_ = True

In [134]:
#@title Get Vocabulary
# vocab char/word [T/F] (default = T)
def vocab(lyrics, char = char_): #might do global char to specify type in beginning
    if char == True:
        vocab = sorted(set(lyrics))
        print(f'{len(vocab)} unique characters in vocabulary')
    else:
        lyrics = [w for w in lyrics.split(' ')]
        vocab = sorted(set(lyrics))
        print(f'{len(vocab)} unique words in vocabulary')
    return lyrics, vocab

lyrics, vocab = vocab(lyrics)

print(type(lyrics)); print(type(vocab))

29 unique characters in vocabulary
<class 'str'>
<class 'list'>


**Vectorize**

In [135]:
#@title Vectorize
# vec char/word [T/F] (default = T)
## keep this structure for now, reduce later
def vec_data(vocab, lyrics, char = char_):
    if char == True: 
        print('character-based')
        char2idx = {u:i for i, u in enumerate(vocab)}
        idx2char = np.array(vocab)
        text_as_int = np.array([char2idx[c] for c in lyrics])
        return char2idx, idx2char, text_as_int

    else:
        word2idx = {u:i for i, u in enumerate(vocab)}
        print('word-based')
        idx2words = np.array(vocab)
        word_as_int = np.array([word2idx[w] for w in lyrics])
        return word2idx, idx2words, word_as_int

In [136]:
# call based on char [T/F]
cw_2idx, idx2_cw, as_int = vec_data(vocab, lyrics, char = char_)

#%%
print(type(cw_2idx)); print(type(idx2_cw)); print(type(as_int))
print(len(cw_2idx)); print(len(idx2_cw)); print(len(as_int))

character-based
<class 'dict'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
29
29
328295


In [137]:
#@title Set Sequence
#define sequence
def seq(lyrics = lyrics, char = char_):
    global seq_length
    global examples_per_epoch
    if char == True:
        seq_length = 100
        examples_per_epoch = len(lyrics)//(seq_length+1)
    else: 
        seq_length = 10
        examples_per_epoch = len(corpus)//(seq_length+1)

#call seq
seq()

print(f'Sequence length set to: {seq_length}')
print(f'Examples per epoch set to: {examples_per_epoch}')

Sequence length set to: 100
Examples per epoch set to: 3250


**Set Model Parameters**

In [138]:
cw_dataset = tf.data.Dataset.from_tensor_slices(as_int)
sequences = cw_dataset.batch(seq_length+1, drop_remainder=True) # generating batches of 10 words each

In [139]:
#further preprossing (duplicate and shift one step below) (helper function)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [140]:
dataset = sequences.map(split_input_target)

# define batch & buffer size
batch_size = 64
buffer_size = 100 #this should be changed with char/word, consider putting it in parameter function

steps_per_epoch =  len(lyrics) // batch_size

#shuffle data
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True) #.prefetch(tf.data.experimental.AUTOTUNE)

# set parameters
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

# check shapes
print(dataset) # should be ((int, int), (int, int)), else run from top

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


**Create Model**

In [141]:
def createModel(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [142]:
if char_ == True:
    model_char = createModel(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size)
    print('Character based Model: ')
    model_char.summary()
else:
    model_word = createModel(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size)
    print('Word based Model: ')
    model_word.summary()

Character based Model: 
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           7424      
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 29)            29725     
Total params: 3,975,453
Trainable params: 3,975,453
Non-trainable params: 0
_________________________________________________________________


**Set Loss Function**

In [143]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


if char_ == False:
    EPOCHS = 20
    start_word = time.time()
    model_word.compile(optimizer='adam', loss=loss)

    #save training checkpoints
    checkpoint_dir = './training_checkpoints_word'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)
    
    history = model_word.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
    tf.train.latest_checkpoint(checkpoint_dir)
    
    model_word = create_model(len(vocab), embedding_dim, rnn_units, batch_size=1)

    model_word.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

    model_word.build(tf.TensorShape([1, None]))
    end_word = time.time()
    print(round(end_word-start_word, 2), 'seconds running time for word-based model')
    
    

elif char_ == True:
    
    EPOCHS = 10
    start_char = time.time()
    model_char.compile(optimizer='adam', loss=loss)
    
    #save training checkpoints
    checkpoint_dir = './training_checkpoints_char'
    
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)
    
    history = model_char.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
    tf.train.latest_checkpoint(checkpoint_dir)
    
    model_char = createModel(len(vocab), embedding_dim, rnn_units, batch_size=1)

    model_char.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

    model_char.build(tf.TensorShape([1, None]))
    end_char = time.time()
    print(round(end_char-start_char, 2), 'seconds running time for char-based model')

else: 
    print('Set char_ to T/F')

Epoch 1/10
50/50 [==============================] - 41s 795ms/step - loss: 3.0889
Epoch 2/10
50/50 [==============================] - 40s 798ms/step - loss: 2.2426
Epoch 3/10
50/50 [==============================] - 40s 792ms/step - loss: 2.0438
Epoch 4/10
50/50 [==============================] - 39s 789ms/step - loss: 1.9157
Epoch 5/10
50/50 [==============================] - 40s 792ms/step - loss: 1.8066
Epoch 6/10
50/50 [==============================] - 40s 797ms/step - loss: 1.7079
Epoch 7/10
50/50 [==============================] - 40s 795ms/step - loss: 1.6195
Epoch 8/10
50/50 [==============================] - 39s 789ms/step - loss: 1.5404
Epoch 9/10
50/50 [==============================] - 40s 800ms/step - loss: 1.4671
Epoch 10/10
50/50 [==============================] - 40s 793ms/step - loss: 1.4012
400.54 seconds running time for char-based model


In [22]:
def generateLyrics(model, startString, temp, num_generate = 30):
    #print("---- Generating lyrics starting with '" + startString + "' ----")

    # Converting our start string to numbers (vectorizing)
    start_string_list =  [w for w in startString.split(' ')]
    input_eval = [cw_2idx[s] for s in start_string_list]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # temp represent how 'conservative' the predictions are. 
        # Lower temp leads to more predictable (or correct) lyrics
        predictions = predictions / temp 
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(' ' + idx2_cw[predicted_id])

    return (startString + ''.join(text_generated))

#save trained model for future use (so we do not have to train it every time we want to generate text)
model_word.save('saved_model.h5') 
print("Example:")
print(generateLyrics(model_word, startString=u"love", temp=0.6))
while (True):
    print('Enter start string:')
    input_str = input().lower().strip()
    print('Enter temp:')
    temp = float(input())
    print(generateLyrics(model_word, startString=input_str, temp=temp))

NameError: name 'model_word' is not defined

In [20]:
def make_song(format,model):
    
    temp = 0.6
    song = []
    
    if model == model_char:
        a,b,c,d,e = 200,300,300,200,300
        generator = char_based_generator
    else: 
        a,b,c,d,e = 20,30,30,20,30
        generator = generateLyrics
        
    intro = generator(model_char, startString=random.choice(most_occur_intro)[0], temp=temp, num_generate = a)
    chorus = generator(model_char, startString=random.choice(most_occur_chorus)[0], temp=temp, num_generate = b)
    bridge = generator(model_char, startString=random.choice(most_occur_bridge)[0], temp=temp, num_generate = c)
    outro = generator(model_char, startString=random.choice(most_occur_outro)[0], temp=temp, num_generate = d)
    
    for tag in format:
        if tag == "I":
            print('[INTRO]\n',intro,'\n')
        elif tag == "V":
            verse = generator(model_char, startString=random. choice(most_occur_verse)[0], temp=temp, num_generate = e)
            print('[VERSE]\n',verse,'\n')
        elif tag == "B":
            print('[BRIDGE]\n',bridge,'\n')
        elif tag == "C":
            print('[CHORUS]\n',chorus,'\n')
        elif tag == "O":
            print('[OUTRO]\n',outro)
        else: 
            raise ValueError('wrong input')


In [25]:
song = make_song("IVCVBCVO",model_char)
song

[INTRO]
 you here home in am we are going to carry you want me the wait. i cull enged me to you the waiting but ev'rybody do not you know i think it is go on the benthow but i am not no don ever have here all you 

[VERSE]
 is bempmony lead me the wanter have been plone to the you one sweet donn and i do not know what a said make me starting then it all got to know something rack of the world do not want me say he will have here it is not the end of the sun and in the meanie conds. we all gotether now a to calling of bea 

[CHORUS]
 to you do not know my love to the thought the world the party called me no i no not no do not know know show my knee when you can not you to be a uppering to the singers off i need so i will not the pain out a seace the ring back to well you will get you one can not going to the way a pain easter you  

[VERSE]
 a hand masic finger dim not every the ligsten but a pooplow now i need it is come so hall hand not girl you know that i have and i do not want to b

**Lyric Generators**



In [24]:
# The prediction loop
def char_based_generator(model, startString, temp, num_generate = 30):
  # Evaluation step (generating text using the learned model)
 
    # Converting our start string to numbers (vectorizing)
    input_eval = [cw_2idx[s] for s in startString]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = temp
 
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2_cw[predicted_id])
 
    return (startString + ''.join(text_generated))
model_char.save('char_based_model.h5') 
print(char_based_generator(model_char, startString=u"hey",temp = 0.6))

hey happy we want me so in. it ev


In [ ]:
def sequence_loss_by_example(logits,
                             targets,
                             weights,
                             average_across_timesteps=True,
                             softmax_loss_function=None,
                             name=None):
  """Weighted cross-entropy loss for a sequence of logits (per example).
  Args:
    logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].
    targets: List of 1D batch-sized int32 Tensors of the same length as logits.
    weights: List of 1D batch-sized float-Tensors of the same length as logits.
    average_across_timesteps: If set, divide the returned cost by the total
      label weight.
    softmax_loss_function: Function (labels-batch, inputs-batch) -> loss-batch
      to be used instead of the standard softmax (the default if this is None).
    name: Optional name for this operation, default: "sequence_loss_by_example".
  Returns:
    1D batch-sized float Tensor: The log-perplexity for each sequence.
  Raises:
    ValueError: If len(logits) is different from len(targets) or len(weights).
  """
      if len(targets) != len(logits) or len(weights) != len(logits):
          raise ValueError("Lengths of logits, weights, and targets must be the same "
                     "%d, %d, %d." % (len(logits), len(weights), len(targets)))
    with ops.name_scope(name, "sequence_loss_by_example",
                      logits + targets + weights):
    log_perp_list = []
    for logit, target, weight in zip(logits, targets, weights):
      if softmax_loss_function is None:
        # TODO(irving,ebrevdo): This reshape is needed because
        # sequence_loss_by_example is called with scalars sometimes, which
        # violates our general scalar strictness policy.
        target = array_ops.reshape(target, [-1])
        crossent = nn_ops.sparse_softmax_cross_entropy_with_logits(
            labels=target, logits=logit)
      else:
        crossent = softmax_loss_function(target, logit)
      log_perp_list.append(crossent * weight)
    log_perps = math_ops.add_n(log_perp_list)
    if average_across_timesteps:
      total_size = math_ops.add_n(weights)
      total_size += 1e-12  # Just to avoid division by 0 for all-0 weights.
      log_perps /= total_size
  return log_perps

In [234]:
#testset is equal to test from the beginning.
#this should work for n-grams. 
#need a big dictionary with the probability for each word. aka. model[word]

def perplexity(testset, model)
    testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

In [244]:
logits = keras.layers.Dense(10)(inputs)

NameError: name 'inputs' is not defined

In [242]:
probabilities = tf.nn.softmax(model_char)

ValueError: Attempt to convert a value (<tensorflow.python.keras.engine.sequential.Sequential object at 0x7fbc9f0f2c10>) with an unsupported type (<class 'tensorflow.python.keras.engine.sequential.Sequential'>) to a Tensor.

In [235]:
perplexity(test, model_char)

TypeError: 'Sequential' object is not subscriptable